In [1]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from geopy.distance import distance
import math

### Prueba consulta de restaurantes 

In [2]:
cluster = Cluster()
session = cluster.connect('challenge')

In [3]:
result = session.execute('select * from restaurants limit 1')

In [4]:
for item in result:
    print(item)

Row(id='55cba2476c522cafdb058fdf', location=[40.7092884, -73.8048161], name='The Smile Of The Beyond', type='Point')


### Extraer un vecindario tipo MultiPolygono
* 55cb9c666c522cafdb053a97

In [5]:
result = session.execute(
    'select * from neighborhoods where id = \'55cb9c666c522cafdb053a97\'')
length = session.execute(
    'select count(*) from neighborhoods where id = \'55cb9c666c522cafdb053a97\''
)

In [6]:
print('Puntos extraidos: ' + str(length[0][0]) + '\n\nImprime puntos')
for row in result:
    print(row)

Puntos extraidos: 271

Imprime puntos
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=0, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73311366098218, -73.96230391870556])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=1, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.7329155516237, -73.96226340489365])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=2, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.732913300829765, -73.96236043096485])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=3, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73287557416924, -73.96285045581647])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=4, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73283805163722, -73.96332992178779])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=5, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[40.73284587606134,

## Encontrar restaurantes en una distancia específica respecto a un punto concreto
* Distancia = 1km
* Punto = [40.6973697290538, -73.94378455587042]

In [7]:
distancia = 1  # km
puntoInicial = [40.6973697290538, -73.94378455587042]  # Latitud, Longitud

In [8]:
result = session.execute('select * from restaurants')
filtrado = []
max_1 = max_2 = 0
for row in result:
    if distance(puntoInicial, row[1]).km <= distancia:
        filtrado.append([
            row[0], row[1], row[2],
            round(distance(puntoInicial, row[1]).km, 3)
        ])

In [9]:
result1 = session.execute('select * from restaurants')
result2 = session.execute('select * from restaurants')
punto_prueba = (max(map(lambda x: abs(x[1][0]),
                        result1)), max(map(lambda x: abs(x[1][1]), result2)))

punto_0 = (0, 0)
punto_prueba_reves = (punto_prueba[1], punto_prueba[0])
print(punto_prueba, punto_prueba_reves)

print(distance(punto_0, punto_prueba).km)

try:
    print(distance(punto_0, punto_prueba_reves).km)
except ValueError as e:
    print(e)

(52.5388779, 157.8887924) (157.8887924, 52.5388779)
13829.132020296587
Latitude must be in the [-90; 90] range.


C:\Users\Zaloon098\anaconda3\lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


In [10]:
print('Numero de restaurantes dentro del límite', len(filtrado))
print(
    f'Imprimir restaurantes a distancia {distancia} desde {puntoInicial}:\n\n')

filtrado.sort(key=lambda item: item[3])
for ident, localizacion, nombre_restaurante, distancia in filtrado:
    print(
        f'El restaurante "{nombre_restaurante}" con coordenadas {localizacion} se encuentra a {distancia}km del punto inicial.\n'
    )

Numero de restaurantes dentro del límite 124
Imprimir restaurantes a distancia 1 desde [40.6973697290538, -73.94378455587042]:


El restaurante "Migdalia Place" con coordenadas [40.698466, -73.944343] se encuentra a 0.131km del punto inicial.

El restaurante "Elisa Cafe" con coordenadas [40.695837, -73.943679] se encuentra a 0.17km del punto inicial.

El restaurante "Crown Fried Chicken" con coordenadas [40.695896, -73.942544] se encuentra a 0.194km del punto inicial.

El restaurante "Genao" con coordenadas [40.699517, -73.944503] se encuentra a 0.246km del punto inicial.

El restaurante "Towers Cafe" con coordenadas [40.699474, -73.94279399999999] se encuentra a 0.248km del punto inicial.

El restaurante "Crown Fried Chicken" con coordenadas [40.6961798, -73.9467093] se encuentra a 0.28km del punto inicial.

El restaurante "Mike'S Pizza" con coordenadas [40.695877, -73.946589] se encuentra a 0.289km del punto inicial.

El restaurante "China City Restaurant" con coordenadas [40.695867,

# Seleccionar los restaurantes que se encuentren el el vecindario: Arden Heights

In [11]:
id_vencindario = '55cb9c666c522cafdb053ac0'

In [12]:
result = session.execute(
    f"select id_restaurant from restaurantsbyneighborhood where id_neighborhood = '{id_vencindario}'"
)

In [13]:
restaurants_ids_str = map(lambda row: f"'{row[0]}'", result)
restaurants_ids_str = ','.join(restaurants_ids_str)
restaurants_ids_str = '(' + restaurants_ids_str + ')'

In [14]:
result  = session.execute('select * from restaurants where id in '+ restaurants_ids_str)

In [15]:
for row in result: 
    print(row)

Row(id='55cba2476c522cafdb053f77', location=[40.55274360000001, -74.1914658], name='Villa Monte Pizzeria', type='Point')
Row(id='55cba2476c522cafdb054221', location=[40.5530453, -74.19261519999999], name='Bagel Deli / Village Green Deli', type='Point')
Row(id='55cba2476c522cafdb0561c9', location=[40.5523287, -74.1910312], name='Genki Sushi', type='Point')
Row(id='55cba2476c522cafdb057bf5', location=[40.5530453, -74.19261519999999], name='Peking Taste', type='Point')
Row(id='55cba2476c522cafdb058a17', location=[40.5530453, -74.19261519999999], name="Moe'S Southwest Griil", type='Point')
Row(id='55cba2476c522cafdb058d37', location=[40.5530453, -74.19261519999999], name='Subway', type='Point')
Row(id='55cba2476c522cafdb058f15', location=[40.5625848, -74.19097219999999], name='Dugout Pub West', type='Point')


In [16]:
cluster.shutdown()